方針として、Deepdreamは諦めて、モデル作成時の精度についての完成度を見せよう。

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
import random
import cv2 

In [2]:
PATH = "../data/data/train_fish/"
# path devided into kinds of image
ALB = "ALB"
BET = "BET"
DOL = "DOL"
LAG = "LAG"
NoF = "NoF"
OTHER = "OTHER"
SHARK = "SHARK"
YFT = "YFT"
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
channnels=3

## データ

データはディレクトリごとに別れていて、合計で3777枚ある。

In [13]:
import os
for path, di, file  in os.walk(PATH):
    CLASS = di
    break
NUM_CLASSES = len(CLASS)
# IMAGE_SIZE = 256
IMAGE_SIZE = 32
IMAGE_PIXELS = IMAGE_SIZE*IMAGE_SIZE*3
learning_rate = 1e-4
max_steps = 200
BATCH_SIZE = 100

# 関数群

In [4]:
# これらコードは、http://kivantium.hateblo.jp/entry/2015/11/18/233834から得られる
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

# 読み込み系
def load_files(path):
    data = []
    save_pa = []
    for pa,_,_ in os.walk(path):
        if pa =='../data/data/train_fish/':
            continue
        save_pa.append(pa)
    for i_c, cat in enumerate(save_pa):
        for _,_,files in os.walk(cat):
            for file in files:
                label = np.zeros(8)
                label[i_c] = 1
                filepath = os.path.join(cat,file)
                data.append([filepath,label])
    return data

# opnecvによる読み込み
def split_image_and_label(data_list,LESS=False):
    '''
    data_list should be [[data,label]...]
    '''
    data = []
    labels = []
    if LESS:
        i=0
        for value,label in data_list:
            i+=1
            data.append(value)
            labels.append(label)
            if i == 1000:
                break
        return data,labels
    else:
        for value,label in data_list:
            data.append(value)
            labels.append(label)
        return data, labels

def convert_image_path_to_data_cv2(path,i):
    try:
#         img2 = cv2.imread(path,0) # 白黒画像で読み込み
        img2 = cv2.imread(path)

        img2 = cv2.resize(img2,dsize=(IMAGE_SIZE,IMAGE_SIZE),interpolation = cv2.INTER_CUBIC)
        np_image_data = np.asarray(img2)
        np_final = np.expand_dims(np_image_data,axis=0)
        np_final = flatten(np_final)
        if i % 1000 == 0:
            print(i)
        #1000 images / 15s
        return np_final
    except:
        print(path)
        print(i)
        return np.zeros([256,256])
    
def flatten(image):
    image = image.flatten().astype(np.float32)/255.0
    return image

# モデル作成

In [5]:
images_placeholder = tf.placeholder(tf.float32, [None, IMAGE_PIXELS])
x_image = tf.reshape(images_placeholder,[-1,IMAGE_SIZE,IMAGE_SIZE,3])  # 32 * 32 * 3 tensorへ変換する
tf.summary.image("input_data",x_image,NUM_CLASSES)
y = tf.placeholder(tf.float32,[None,NUM_CLASSES])
keep_prob = tf.placeholder("float")

with tf.name_scope('conv1') as scope:
    W_conv1 = weight_variable([5,5,3,32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)

with tf.name_scope('pool1') as scope:
    h_pool1 = max_pool_2x2(h_conv1)

with tf.name_scope('conv2') as scope:
    W_conv2 = weight_variable([5,5,32,64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)

with tf.name_scope('pool2') as scope:
    h_pool2 = max_pool_2x2(h_conv2)
    # h_pool2 has (?, 64, 64, 64) shape
#全結合
with tf.name_scope('zenketsu1') as scope:
    W_fc1 = weight_variable([64*64*1,1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2,[-1,64*64*1]) #3136 defined here
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

with tf.name_scope('zenketsu2') as scope:
    W_fc2 = weight_variable([1024,NUM_CLASSES])
    b_fc2 = bias_variable([NUM_CLASSES])

with tf.name_scope('softmax') as scope:
    out = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

#loss, train, を追記
with tf.name_scope('loss'):
    loss = -tf.reduce_sum(y*tf.log(out))
    # ログ出力
    tf.summary.scalar("loss",loss)
    
with tf.name_scope('train'):
    train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(out,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    # 精度出力
    tf.summary.scalar("accuracy",accuracy)

とりあえず、推論までを平がきした。ネームスコープがあるから、テンサーボードで処理を逐次出力できる。
また、loss, accuracyはログ出力させるようにした。

# データ読み込み

In [6]:
DATA = load_files(PATH)
import csv
with open("../data/meta.csv",'w') as f:
    writer = csv.writer(f,lineterminator='\n')
    writer.writerow(DATA)

使用しているデータの頻度を掛けないといけないかも。それ以外の解放だと、入れたデータについて、albが出る確率が0.5以上であるかどうか。

In [7]:
random.shuffle(DATA)
num_train = int(len(DATA) * 0.8)
train = DATA[:num_train]
test = DATA[num_train:]
#trainデータを三倍にかさ増しする。
train = train*5

精度が0.11で固定されていたので、学習量不足と判断し、データ数を増やした。

In [8]:
# 結果の考察のためalbとdblのみで処理してみる。
def take(data,num):
    reserved = []
    for v in data:
        if np.argmax(v[1]) == num:
            reserved.append(v)
    return reserved

In [24]:
alb_test = take(test,2)
alb_test_image, alb_test_label = split_image_and_label(alb_test,False)
alb_test_image = [convert_image_path_to_data_cv2(path,i) for i,path in enumerate(alb_test_image)]
print("Images are successfully converted.")
    
dlb_test = take(test,3)
dlb_test_image, dlb_test_label = split_image_and_label(dlb_test,False)
dlb_test_image = [convert_image_path_to_data_cv2(path,i) for i,path in enumerate(dlb_test_image)]
print("Images are successfully converted.")  

nof_test = take(test,0)
nof_test_image, nof_test_label = split_image_and_label(nof_test,False)
nof_test_image = [convert_image_path_to_data_cv2(path,i) for i,path in enumerate(nof_test_image)]
print("Images are successfully converted.")  

0
Images are successfully converted.
0
Images are successfully converted.
0
Images are successfully converted.


# Tensorflow実行

In [14]:
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter("../log",sess.graph)
    init = tf.global_variables_initializer()
    sess.run(init)
    summary_op = tf.summary.merge_all()
    
    #for debugging True
    train_image, train_label = split_image_and_label(train,False)
    test_image,test_label = split_image_and_label(test,False) 
    print("The number of train images: {}".format(len(train_image)))
    print("The number of test images: {}".format(len(test_image)))

    
    train_image = [convert_image_path_to_data_cv2(path,i) for i,path in enumerate(train_image)]
    test_image = [convert_image_path_to_data_cv2(path,i) for i,path in enumerate(test_image)]
    #takes 20 mins
    print("Images are successfully converted.")
    
    

    # 訓練の実行
    for step in range(max_steps):
        for i in range(int(len(train_image)/BATCH_SIZE)):
            # batch_size分の画像に対して訓練の実行
            batch = BATCH_SIZE*i
            # feed_dictでplaceholderに入れるデータを指定する
            sess.run(train_step, feed_dict={
              images_placeholder: train_image[batch:batch+BATCH_SIZE],
              y: train_label[batch:batch+BATCH_SIZE],
              keep_prob: 0.5})

        # 1step終わるたびに精度を計算する
        train_accuracy = sess.run(accuracy, feed_dict={
            images_placeholder: train_image,
            y: train_label,
            keep_prob: 1.0})
        print("step %d, training accuracy %g"%(step, train_accuracy))

        # 1steps終わるごとにログを取得する

        print("logging")
        summary_str = sess.run(summary_op, feed_dict={
            images_placeholder: test_image, 
            y: test_label,
            keep_prob: 1.0
        })
        summary_writer.add_summary(summary_str, step)

    # 最終的なモデルを保存
    saver = tf.train.Saver()
    save_path = saver.save(sess, "../data/model/second_model/model.ckpt")
    
    
    # 訓練が終了したらテストデータに対する精度を表示
    print("test accuracy %g"%sess.run(accuracy, feed_dict={
        images_placeholder: test_image,
        y: test_label,
        keep_prob: 1.0}))
    
   
    print("test accuracy ALB:%g"%sess.run(accuracy,feed_dict={
         images_placeholder: alb_test_image,
        y: alb_test_label,
        keep_prob: 1.0
    }))
    
    print("test accuracy DLB:%g"%sess.run(accuracy,feed_dict={
         images_placeholder: dlb_test_image,
        y: dlb_test_label,
        keep_prob: 1.0
    }))


The number of train images: 15105
The number of test images: 756
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
0
Images are successfully converted.
step 0, training accuracy 0.0264813
logging
step 1, training accuracy 0.0271433
logging
step 2, training accuracy 0.0278054
logging
step 3, training accuracy 0.0291294
logging
step 4, training accuracy 0.0314465
logging
step 5, training accuracy 0.0340947
logging
step 6, training accuracy 0.0370738
logging
step 7, training accuracy 0.041046
logging
step 8, training accuracy 0.0450182
logging
step 9, training accuracy 0.0526316
logging
step 10, training accuracy 0.0645482
logging
step 11, training accuracy 0.0830851
logging
step 12, training accuracy 0.114532
logging
step 13, training accuracy 0.147633
logging
step 14, training accuracy 0.184376
logging
step 15, training accuracy 0.234028
logging
step 16, training accuracy 0.276068
logging
step 17, training accuracy 0.312479
logging
step 18, training accu

step 181, training accuracy 0.473684
logging
step 182, training accuracy 0.473684
logging
step 183, training accuracy 0.474015
logging
step 184, training accuracy 0.475008
logging
step 185, training accuracy 0.475008
logging
step 186, training accuracy 0.475339
logging
step 187, training accuracy 0.475339
logging
step 188, training accuracy 0.476332
logging
step 189, training accuracy 0.476332
logging
step 190, training accuracy 0.476332
logging
step 191, training accuracy 0.476332
logging
step 192, training accuracy 0.476663
logging
step 193, training accuracy 0.476663
logging
step 194, training accuracy 0.476663
logging
step 195, training accuracy 0.476663
logging
step 196, training accuracy 0.477325
logging
step 197, training accuracy 0.476994
logging
step 198, training accuracy 0.477656
logging
step 199, training accuracy 0.477987
logging
test accuracy 0.473545
test accuracy ALB:0.979472
test accuracy DLB:0


In [17]:
nof_test = take(test,0)
oth_test = take(test,1)

In [20]:
len(alb_test)

341

In [21]:
len(test)

756

In [22]:
len(alb_test)/ len(test)

0.45105820105820105

チャンスレートは0.451

In [25]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('../data/model/second_model/model.ckpt.meta')
    saved_path = '../data/model/second_model/model.ckpt'
    saver.restore(sess,saved_path)
    print("restored")
    print("test accuracy DLB:%g"%sess.run(accuracy,feed_dict={
         images_placeholder: nof_test_image,
        y: nof_test_label,
        keep_prob: 1.0
    }))

INFO:tensorflow:Restoring parameters from ../data/model/second_model/model.ckpt
restored
test accuracy DLB:0.0111111


### 編集用